# Model Problem Prototype

The purpose of this notebook is to demonstrate the use of `regen` module functions.  The purpose of the `weak_form()` function is to take a partial differential equation and return the same equation in weak form.  One of the reasons the weak form of a PDE is advantageous for a finite element formulation is that second derivatives can be reduced to first derivatives by employing integration by parts. This puts fewer restrictions on the smoothness of the solution variable(s).  However, applying the integration-by-parts formula to a PDE in a symbolic manipulation package such as `sympy` is non-trivial.

The purpose of the `galerkin()` function is to return an expression (or expressions) in which the input expression(s) have been substituted to use a Galerkin expansion approximation.

## The Model Problem

We begin by importing functions and classes we will need from the Python module `sympy`.  Calling the `init_printing()` function will give us nicely formatted output mathematics within the notebook.  The `regen` module is the Python module being developed here, whose goal is to start with a practical expression of a PDE (in LaTeX, for example, or using `sympy` syntax, as we do here) and from that, generate a `ResidualEvaluator` class, suitable for use in Albany.

In [ ]:
from sympy import init_printing, Symbol, Function, Eq, Derivative
import regen
init_printing()

The model problem we will be using for demonstration purposes comes from *Finite Elements: An Introduction, Volume I* by Eric B. Becker, Graham F. Carey, and J. Tinsley Oden.  In chapter 1 of that textbook, the authors develop a finite element method in the context of a model problem

$$
\begin{array}{ccc}
    -u'' + u = x, & 0 < x < 1 \\
    u(0) = 0,     & u(1) = 0
\end{array}
$$

where $u$ is our solution variable and the prime symbol $'$ indicates differention with respect to $x$.

To define our equation using `sympy`, we need to define `x` as a `Symbol` and `u` as a `Function` of `x`, then use the `Eq` and `Derivative` classes to define the equation:

In [ ]:
x = Symbol("x")
y = Symbol("y")
u = Function("u")(x)
model_prob = Eq(-Derivative(u,x,2) + u, x)
model_prob

## Weak Form

Now the goal is to obtain the weak form of our model problem.  The `weak_form` function takes an `Eq` object as its first argument, the test function as its second argument, which we name `v`.  Note that through inspection, the `weak_form` function detects the solution variables, in this case `(u(x),)` and the domain variable `x`.

In [ ]:
v = Function("v")(x)
weak_form = regen.weak_form(model_prob, v)
weak_form

The weak form is returned in the form of a `sympy.Expr` rather than a `sympy.Eq`, with the implication that the expression sums to zero.  Notice that the term involving the second derivative of $u$ has been replaced by two terms that involve the first derivatives of $u$ and $v$.

Open questions:
1. How much work will be involved to upgrade the `regen.weak_form()` function to handle higher dimensions?  The `sympy` objects already handle multi-dimensional objects, but the integration, including integration by parts, will have to be upgraded to handle higher dimensions.
1. The `regen.weak_form()` function can handle one or more equations, but has not yet been tested for multiple equations.
1. The work here was done in terms of indefinite integrals, but in practice, we need to include the boundary conditions.  The model problem here has Dirichlet boundary conditions and should be easy to handle, but more complicated boundary conditions could add complexity.

## Galerkin Approximation

We now want to substitute an approximation for $u$ into our model problem.  Specifically, we are going to change the model problem from solving for $u_h$ instead of $u$, where

$$
u_h(x) = \sum_{i=0}^{N-1} u_i \phi_i(x)
$$

and the test functions are given by

$$
v(x) = \phi_j(x)
$$

and the ${\phi_i(x)}$ are our basis functions of dimension $N$.  Note that `sympy` supports `IndexedBase` objects and `Function` objects, but not `Function`s that are indexed (at least not that I have figured out yet).  My workaround for this is to define `phi` as a `Function`, with two arguments, `i` (or `j`) and `x`.  This results in the mathematical notation $\phi(i,x)$ rather than the more traditional $\phi_i(x)$.

The `regen.galerkin()` function takes is its first argument an expression or a sequence of expressions, in this case our `weak_form` expression.  For the second argument, it needs to know the test function we used to obtain the weak form, because this function does not get expanded as a series.  The third argument is the basis function for the Galerkin approximation.  It should be constructed without arguments, neither index nor domain variable, as the `regen.galerkin()` function will add these.

In [ ]:
phi = Function('phi')
galerkin = regen.galerkin(weak_form, v, phi)
galerkin

## Apply Quadrature

We now wish to convert the integrals to summations by applying Gaussian quadrature, which can be done by the `regen.quadrature()` function:

In [ ]:
expr = regen.quadrature(galerkin)
expr

The final step before code generation is to convert derivatives such as $\partial \phi(i,x_k)/\partial x$ to a function such as $\phi_x(i,x_k)$, using the `regen.substitute_derivatives()` function:

In [ ]:
regen.substitute_derivatives(expr)

At this point in the process, we have hit __[the substitution bug](http://localhost:8888/notebooks/Substitution%20Bug.ipynb)__, which prevents proper processing (the above expression should have regular functions substituted for the derivatives).  As a temporary workaround, I will `print(expr)`, cut-and-paste, and then modify the expression to be what we need it to be.

In [ ]:
print(expr)

In [ ]:
from sympy import Idx, IndexedBase, Sum, Symbol, Integral
i     = Idx('i')
j     = Idx('j')
k     = Idx('k')
N     = Symbol('N', integer=True)
N_k   = Symbol('N_k', integer=True)
ui    = IndexedBase('u')
xk    = IndexedBase('x')
w     = IndexedBase('w')
phi_x = Function('phi_x')
discrete = -phi(j, xk[k])*Sum(phi_x(i, xk[k])*ui[i], (i, 0, N - 1)) - Sum(x*phi(j, xk[k])*w[k], (k, 0, N_k - 1)) + \
Sum(phi(j, xk[k])*w[k]*Sum(phi(i, xk[k])*ui[i], (i, 0, N - 1)), (k, 0, N_k - 1)) + \
Sum(phi_x(j, xk[k])*w[k]*Sum(phi(i, xk[k])*ui[i], (i, 0, N - 1)), (k, 0, N_k - 1))
discrete